## Introduction to ```spaCy```

There are a number of different NLP frameworks that you're likely to encounter. The most popular and widely-used of these are:

- ```NLTK``` (Natural Language Toolkit, old-school)
- ```UDPipe``` (Neural network based, fast and light, but not super accurate)
- ```CoreNLP``` and ```stanza``` (Created by the team at Stanford; academically robust)
- ```spaCy``` production-ready, well-documented, state-of-the-art

We'll be working with ```spaCy``` in this module, primarily because it's easy and intuitive, and also scales well.

First thing we need to do is install ```spaCy``` and the language model that we want to use.

From the command line, you should first make sure to run the setup script to install requirements:

```shell 
bash setup.sh
```

## Initializing ```spaCy```

The first thing we need to do is import ```spaCy``` __and__ the language model that we want to use.

Note that, if you want to use different langauges you want to use different language models.

In [6]:
# create a spacy NLP class
import spacy
nlp = spacy.load("en_core_web_md")

With the model now loaded, we can begin to do some very simple NLP tasks.

Here, we create a spaCy object and assign it to the variable ```nlp```. This is the NLP pipeline that will do all our heavy lifting, using the trained model we've specified.

Below, you can see what the pipeline does with a bit of sample text. Passing text to the nlp object gives us access to a bunch of properties, including tokens (words), parts of speech, named entities, and so on. Here's two of them, tokens and entities. These objects, in turn, have certain methods attached to them. A full outline of available methods can be found in the spaCy docs.

In this case, for all token objects, let's return the token itself (```token.text```); its part-of-speech tag (```token.pos_```); and the grammatical dependency relations between the tokens (```token.dep_```).


In [8]:
# a single sentence example
input_string = "My name is Ross and I come from Scotland."

In [10]:
# create a new Doc object
doc = nlp(input_string)

In [23]:
# checking type of object
type(doc)

spacy.tokens.doc.Doc

In [24]:
# printing the object - it gives the string
print(doc)

My name is Ross and I come from Scotland.


__Tokenize__

In [25]:
# tokenizing text
for token in doc:
    print(token.text)
# using the .text gives all the tokens thir own attributes

My
name
is
Ross
and
I
come
from
Scotland
.


__Trying some more attributes__

In [32]:
# find parts-of-speech and grammatical relations
for token in doc:
    print(token.i, token.text, token.pos_, token.dep_, token.morph)
# the .i gives all tokens their number in the string
# the .pos gives all tokens a number according to their word type - e.g. 96
# adding .pos_ instead gives the word type - e.g. PRON, VERB
# the .dep_ tells us more about the structure
# the .morph gives us morphological information


0 My PRON poss Number=Sing|Person=1|Poss=Yes|PronType=Prs
1 name NOUN nsubj Number=Sing
2 is AUX ROOT Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin
3 Ross PROPN attr Number=Sing
4 and CCONJ cc ConjType=Cmp
5 I PRON nsubj Case=Nom|Number=Sing|Person=1|PronType=Prs
6 come VERB conj Tense=Pres|VerbForm=Fin
7 from ADP prep 
8 Scotland PROPN pobj Number=Sing
9 . PUNCT punct PunctType=Peri


__NER__

Extracting named entities from a ```spaCy``` doc requires an extra step, but nothing too challenging:

In [35]:
# extracting NERs
for ent in doc.ents:
    print(ent.text, ent.label_)
# it will show the named entities - e.g. Ross, Scotland
# adding ent.label will show the type of entity - e.g. PERSON, GPE(geopolitical entity)


Ross PERSON
Scotland GPE


__Questions:__ 

1. What range of linguistic features is available beyond what we're looking at here? 
2. Are the same range of features available for all languages? Compare e.g. English and Danish.

## Count distribution of linguistic features

__Create doc object__

In [46]:
# load a text file
import os
filename = os.path.join("..", "data", "nicola.txt")

In [47]:
with open(filename, "r", encoding="utf-8") as file:
    text = file.read()

In [52]:
# create a doc object
doc =nlp(text)
print(doc)

Nicola Sturgeon is to quit as Scotland’s first minister in a shock decision. A source confirmed Sturgeon, who has been the country’s longest-serving first minister, will announce her resignation at 11am on Wednesday in Edinburgh. The Scottish National party leader has suffered a series of political setbacks recently, including the UK supreme court defeat of her plans for a fresh independence referendum and a damaging row over a transgender double rapist being sent to a female jail.



In [49]:
# create empty list
entities = []

# add each entity to list
for ent in doc.ents:
    entities.append(ent.text)

In [53]:
# showing the unique set of entities in my text
print(set(entities))

{'Nicola Sturgeon', 'Scottish National', 'Edinburgh', 'Wednesday', 'first', 'UK', 'Scotland', 'Sturgeon', '11am on'}


In [57]:
# count number of adjectives
adjective_count = 0
for token in doc:
    if token.pos_ == "ADJ":
        adjective_count += 1

In [58]:
adjective_count

7

__Relative frequency__

In [61]:
# find the relative frequency per 10,000 words
relative_freq = (adjective_count/len(doc)) * 10000

In [64]:
# rounding the result down to x number of decimals
round(relative_freq, 2)

786.52

## Creating neater outputs using ```pandas```

At the moment, all of our output from ```spaCy``` is in the form of lists. If we want to save these, it probably makes sense to have them saved in a more transferable format, such as CSV files or JSONs.

One very easy way to do this with Python is by using the dataframe library ```pandas```.

In [65]:
import pandas as pd

In [66]:
# create spaCy doc
doc = nlp(input_string)

In [68]:
# appending two things at the same time (tibbles)
annotations = []
for token in doc:
    annotations.append((token.text, token.pos_))

In [69]:
annotations

[('My', 'PRON'),
 ('name', 'NOUN'),
 ('is', 'AUX'),
 ('Ross', 'PROPN'),
 ('and', 'CCONJ'),
 ('I', 'PRON'),
 ('come', 'VERB'),
 ('from', 'ADP'),
 ('Scotland', 'PROPN'),
 ('.', 'PUNCT')]

In [72]:
# spaCy doc to pandas dataframe and changing column names
data = pd.DataFrame(annotations, columns=["Token", "POS"])

In [73]:
data

,Token,POS
0,My,PRON
1,name,NOUN
2,is,AUX
3,Ross,PROPN
4,and,CCONJ
5,I,PRON
6,come,VERB
7,from,ADP
8,Scotland,PROPN
9,.,PUNCT


In [74]:
# save dataframe
outpath = os.path.join("..", "output", "annotations.csv")
data.to_csv(outpath)

## Assignment 1

Spend some time exploring and familiarizing yourself with ```spaCy``` and ```pandas```. We'll come back to them quite a lot through this semester, so it will help to have a solid handle on how they function.

When you are ready, head over to [Assignment 1](https://classroom.github.com/a/PdNi7nPv) which takes some of the skills you've learned last week and today. The task will be to count how many times certain linguistic features occur accross different documents, and to save those results in a clear and easy-to-understand way.